In [ ]:
# Import Required Libraries
import sys
import os
import json
import time
import glob
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

sys.path.insert(0, 'src')
from analyze.Data import Data
from datastruct.Network import Network
from methods.lasso import Lasso
from methods.lsco import LSCO
from methods.tigress import TIGRESS
from analyze.CompareModels import CompareModels
from bootstrap.nb_fdr import NetworkBootstrap

In [ ]:
# Configuration
OUTPUT_DIR = 'benchmark_results'
DATASET_ROOT = os.path.expanduser('../GeneSPIDER2/data/gs-datasets/N50')
NETWORK_ROOT = os.path.expanduser('../GeneSPIDER2/data/gs-networks')
ZETAVEC = np.logspace(-6, 0, 30)
N_INIT = 10
N_BOOT = 10
FDR = 5

# Create output directory
output_dir = Path(OUTPUT_DIR)
output_dir.mkdir(exist_ok=True)

print("Configuration set.")

In [ ]:
# Helper Functions
def run_comparison_analysis(true_net, inferred_net):
    comp = CompareModels(true_net, Network(inferred_net))
    
    try:
        from sklearn.metrics import roc_auc_score
        true_flat = (true_net.A != 0).astype(float).flatten()
        inferred_flat = np.abs(inferred_net.flatten())
        
        if len(np.unique(true_flat)) > 1 and np.sum(inferred_flat) > 0:
            auroc = roc_auc_score(true_flat, inferred_flat)
        else:
            auroc = 0.5
    except:
        auroc = 0.5
    
    return {
        'f1_score': comp.F1[0] if len(comp.F1) > 0 else 0.0,
        'auroc': auroc,
        'precision': comp.pre[0] if len(comp.pre) > 0 else 0.0,
        'recall': comp.sen[0] if len(comp.sen) > 0 else 0.0,
        'mcc': comp.MCC[0] if len(comp.MCC) > 0 else 0.0
    }

def find_network_file(network_dir, network_id):
    matches = list(Path(network_dir).rglob(f"*{network_id}*.json"))
    if matches:
        return matches[0]
    
    patterns = [
        f"**/*ID{network_id}.json",
        f"**/*ID{network_id}*",
        f"**/*{network_id}.json",
        f"**/*{network_id}*"
    ]
    
    for pattern in patterns:
        matches = list(Path(network_dir).rglob(pattern))
        if matches:
            return matches[0]
    
    return None

def run_standard_method(method_name, data, zetavec):
    start_time = time.time()
    
    try:
        if method_name == 'LASSO':
            A_3d, _ = Lasso(data, alpha_range=zetavec)
            A_final = A_3d[:, :, -1]
        elif method_name == 'LSCO':
            A_3d, _ = LSCO(data, threshold_range=zetavec)
            A_final = A_3d[:, :, -1]
        elif method_name == 'TIGRESS':
            A_final = TIGRESS(data)
        else:
            raise ValueError(f"Unknown method: {method_name}")
        
        exec_time = time.time() - start_time
        return A_final, exec_time, None
    
    except Exception as e:
        exec_time = time.time() - start_time
        return None, exec_time, str(e)

def run_nestboot_method(method_name, data, net, zetavec, n_init, n_boot, fdr, seed=42):
    np.random.seed(seed)
    nb = NetworkBootstrap()
    start_time = time.time()
    
    def inference_method(dataset, zetavec=None):
        if method_name == 'LASSO':
            A_3d, _ = Lasso(dataset, alpha_range=zetavec)
            return A_3d
        elif method_name == 'LSCO':
            A_3d, _ = LSCO(dataset, threshold_range=zetavec)
            return A_3d
        else:
            raise ValueError(f"Unknown method: {method_name}")
    
    results = nb.run_nestboot(
        dataset=data,
        inference_method=lambda ds, **kwargs: inference_method(ds, zetavec),
        nest_runs=n_init,
        boot_runs=n_boot,
        seed=seed,
        method_kwargs={}
    )
    
    exec_time = time.time() - start_time
    
    binary_network = np.zeros((data.data.N, data.data.N))
    for idx, (gene_i, gene_j) in enumerate(zip(results.gene_i, results.gene_j)):
        i = int(gene_i.split('_')[1])
        j = int(gene_j.split('_')[1])
        binary_network[i, j] = results.xnet[idx]
    
    metrics = run_comparison_analysis(net, binary_network)
    
    result = {
        'method': f'{method_name}+NestBoot',
        'n_init': n_init,
        'n_boot': n_boot,
        'fdr': fdr,
        'time': exec_time,
        'auroc': metrics['auroc'],
        'f1': metrics['f1_score'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'density': np.sum(binary_network != 0) / binary_network.size,
        'support_threshold': results.support,
        'fp_rate': results.fp_rate
    }
    
    return result

print("Helper functions defined.")

In [ ]:
# Find Datasets
dataset_files = sorted(glob.glob(os.path.join(DATASET_ROOT, "*.json")))
print(f"Found {len(dataset_files)} N50 datasets.")

# Initialize results
all_results = []
results_file = output_dir / 'n50_benchmark_results.csv'

In [ ]:
# Run Benchmark
processed_count = 0

for dataset_path in dataset_files[:5]:  # Limit to first 5 for testing
    dataset_filename = os.path.basename(dataset_path)
    print(f"\nProcessing {dataset_filename}")
    
    try:
        data = Data.from_json_file(dataset_path)
        
        # Extract network ID
        with open(dataset_path, 'r') as f:
            json_data = json.load(f)
        network_id = json_data['obj_data']['network'].split('-ID')[-1]
        
        network_path = find_network_file(NETWORK_ROOT, network_id)
        if not network_path:
            print(f"Network not found for {network_id}")
            continue
        
        net = Network.from_json_file(str(network_path))
        
        # Run methods
        methods = ['TIGRESS', 'LASSO', 'LSCO']
        nestboot_methods = ['LASSO', 'LSCO']
        
        for method in methods:
            print(f"Running {method}...")
            inferred_net, exec_time, error = run_standard_method(method, data, ZETAVEC)
            if inferred_net is not None:
                metrics = run_comparison_analysis(net, inferred_net)
                result = {
                    'dataset': dataset_filename,
                    'network': os.path.basename(str(network_path)),
                    'method': method,
                    'execution_time': exec_time,
                    'f1_score': metrics['f1_score'],
                    'auroc': metrics['auroc'],
                    'precision': metrics['precision'],
                    'recall': metrics['recall'],
                    'mcc': metrics['mcc'],
                    'density': np.sum(inferred_net != 0) / inferred_net.size
                }
                all_results.append(result)
            else:
                print(f"{method} failed: {error}")
        
        for method in nestboot_methods:
            print(f"Running {method}+NestBoot...")
            result = run_nestboot_method(method, data, net, ZETAVEC, N_INIT, N_BOOT, FDR)
            result['dataset'] = dataset_filename
            result['network'] = os.path.basename(str(network_path))
            all_results.append(result)
        
        processed_count += 1
        
        # Save intermediate results
        df_results = pd.DataFrame(all_results)
        df_results.to_csv(results_file, index=False)
    
    except Exception as e:
        print(f"Error processing {dataset_filename}: {e}")
        import traceback
        traceback.print_exc()

print(f"Processed {processed_count} datasets.")
print(f"Results saved to {results_file}.")

In [ ]:
# Summary
if all_results:
    df_results = pd.DataFrame(all_results)
    print("\nSummary by Method:")
    for method in df_results['method'].unique():
        method_results = df_results[df_results['method'] == method]
        print(f"{method}:")
        print(f"  F1: {method_results['f1_score'].mean():.3f} ± {method_results['f1_score'].std():.3f}")
        print(f"  AUROC: {method_results['auroc'].mean():.3f} ± {method_results['auroc'].std():.3f}")
        print(f"  Time: {method_results['execution_time'].mean():.1f} ± {method_results['execution_time'].std():.1f}s")